In [62]:
!pip install clickhouse_driver
!pip install faker


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
from time import time
from uuid import uuid4


from clickhouse_driver import Client

from faker import Faker

In [97]:
fake: Faker = Faker()

client = Client(host='localhost',
                port=9000,
                user='admin',
                password='123')

In [ ]:
client.execute(\
    '''
    CREATE TABLE IF NOT EXISTS player_progress
    (user_id UUID,
     movie_id UUID,
     event_dt DateTime,
     view_progress Int64,
     movie_duration Int64
    )
    ENGINE MergeTree() 
    ORDER BY (event_dt);
    '''
)

client.execute('SHOW TABLES from default')


[('player_progress',)]

In [99]:
BATCH_SIZE: int = 1000
BATCHES: int = 50
TOTAL_RECORDS: float = BATCH_SIZE * BATCHES

start_time: float = time()

INSERT_SQL_QUERY = '''
    INSERT INTO player_progress
    (user_id, movie_id, event_dt, view_progress, movie_duration) VALUES
    '''

for _ in range(BATCHES):
    rows_to_insert = [
            (uuid4(),
             uuid4(),
             fake.date_time_between(start_date="-1y", end_date="now"),
             fake.random_int(min=0, max=1000),
             fake.random_int(min=1001, max=5000))
            for _ in range(BATCH_SIZE)
        ]
    client.execute(query=INSERT_SQL_QUERY, params=rows_to_insert)

insertion_time: float  = time() - start_time

insertion_speed: float  = round(TOTAL_RECORDS / insertion_time, 2)

print('Insertion speed: {:,} records/sec'.format(insertion_speed))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Insertion speed: 5,571.52 records/sec


In [101]:
get_rows_count_sql_query = 'SELECT COUNT(*) FROM player_progress'
rows_count = client.execute(get_rows_count_sql_query)[0][0]

FULL_DATA_SQL_QUERY = 'SELECT * FROM player_progress '
TRIES_COUNT: int = 5
start_time: float = time()
for _ in range(TRIES_COUNT):
    client.execute(query=FULL_DATA_SQL_QUERY)

reading_time: float = time() - start_time
rows_per_second: float = round(rows_count / (reading_time / TRIES_COUNT), 2)

print('Reading speed: {:,} records/sec'.format(rows_per_second))

Reading speed: 77,558.81 records/sec
